In [46]:
import sys
import math

from pyspark.sql.functions import log10
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('pj2').getOrCreate()
sc = spark.sparkContext

In [47]:
def func1(line):
    (year, l1) = line
    temp = []
    for i in l1:
        temp.append((year, i))
    return temp

file = sc.textFile(name=r'tiny-doc.txt', minPartitions=5)
stop_file = sc.textFile(r'stopwords.txt')

# stop_set = set(stop_file.collect())         # 记录stop_word的set
stop_list = stop_file.collect()

temp1 = file.map(lambda line: line.split(","))             # 先分date和term两部分
temp2 = temp1.map(lambda x: (x[0][:4], set(x[1].split())))              # (year,word_list)

# temp2 = temp1.map(lambda x: (x[date][:4], set([x[terms]])-stop_set))             # 只取年份，set使每行value集合不重复并求差集
temp3 = temp2.flatMap(func1).filter(lambda x: True if len(x)!=0 else False)        # 验证不为空
temp4 = temp3.filter(lambda x:True if x[1] not in stop_list else False)            # 剔除stop_word            （year, word）
# temp4.collect()

In [49]:
num_of_year = temp4.keys().distinct().count()
num_of_year

3

In [52]:
# tf
from operator import add

pairs = temp4.map(lambda word:(word,1))
tf1 = pairs.reduceByKey(add)          # ((year,word),count)
tf2 = tf1.map(lambda x:(x[0][1], (x[0][0], x[1])))      # (word, (year, total_count))
tf2.collect()

[('fails', ('2003', 1)),
 ('insurance', ('2003', 2)),
 ('withstand', ('2004', 1)),
 ('cuban', ('2004', 1)),
 ('coronavirus', ('2020', 4)),
 ('streams', ('2020', 1)),
 ('china', ('2020', 2)),
 ('fed', ('2003', 1)),
 ('eels', ('2004', 2)),
 ('vows', ('2004', 1)),
 ('learnt', ('2020', 1)),
 ('coronanomics', ('2020', 1)),
 ('chinese', ('2020', 1)),
 ('remess', ('2020', 1)),
 ('swine', ('2020', 1)),
 ('levy', ('2003', 1)),
 ('introduce', ('2003', 1)),
 ('cowboys', ('2004', 2)),
 ('economy', ('2020', 2)),
 ('community', ('2020', 1)),
 ('test', ('2020', 1)),
 ('bear', ('2020', 1)),
 ('aid', ('2020', 1)),
 ('apartment', ('2020', 1)),
 ('position', ('2003', 1)),
 ('secure', ('2003', 1)),
 ('council', ('2003', 3)),
 ('welcomes', ('2003', 2)),
 ('decision', ('2003', 1)),
 ('opp', ('2003', 1)),
 ('socialism', ('2004', 1)),
 ('bush', ('2004', 1)),
 ('kits', ('2020', 1)),
 ('selling', ('2020', 1)),
 ('classes', ('2020', 1)),
 ('foriegn', ('2020', 1)),
 ('guard', ('2020', 1)),
 ('pig', ('2020', 1)),


In [31]:
# idf; The number of years in D is already known
# how to get the number of years having t
ty = temp4.map(lambda x:(x[1],x[0])).groupByKey().mapValues(lambda x:len(set(x)))      # (word,count)
# ty.collect()
idf = ty.map(lambda x:(x[0],math.log10(num_of_year/x[1])))
# idf.collect()

[('secure', 0.47712125471966244),
 ('chief', 0.47712125471966244),
 ('fails', 0.47712125471966244),
 ('council', 0.47712125471966244),
 ('decision', 0.47712125471966244),
 ('breakthrough', 0.47712125471966244),
 ('introduce', 0.47712125471966244),
 ('eels', 0.47712125471966244),
 ('coronavirus', 0.47712125471966244),
 ('test', 0.47712125471966244),
 ('selling', 0.47712125471966244),
 ('campbell', 0.47712125471966244),
 ('streams', 0.47712125471966244),
 ('bear', 0.47712125471966244),
 ('guard', 0.47712125471966244),
 ('flu', 0.47712125471966244),
 ('blocks', 0.47712125471966244),
 ('position', 0.47712125471966244),
 ('executive', 0.47712125471966244),
 ('fed', 0.47712125471966244),
 ('opp', 0.47712125471966244),
 ('national', 0.47712125471966244),
 ('cowboys', 0.47712125471966244),
 ('socialism', 0.47712125471966244),
 ('vows', 0.47712125471966244),
 ('learnt', 0.47712125471966244),
 ('home', 0.47712125471966244),
 ('classes', 0.47712125471966244),
 ('china', 0.47712125471966244),
 ('a

In [42]:
# A.join(B): 要求key相同，Values变为(A.value(),B.value())
weight1 = tf2.join(idf)       # e.g. 'fails', (('2003', 1), 0.47712125471966244
# weight1.collect()
# 需要转换为 key: year ; values: 'word,total_count'
weight2 = weight1.map(lambda x:(x[1][0][0],x[0],x[1][0][1]*x[1][1])).sortBy(lambda x:(x[0],-x[2],x[1]))
# weight2.collect()          # e.g. ('2003', 'council', 1.4313637641589874),
weight3 = weight2.map(lambda x:(x[0],x[1]+','+str(round(x[2],6)))).groupByKey().mapValues(lambda x:list(x))
# weight3.collect()            # e.g. (year,['word+count1','word+count2'...])
# 取前k个
k = int(3)
weight4 = weight3.map(lambda x:(x[0],';'.join(x[1][:k]))).sortBy(lambda x:x[0]).map(lambda x:'\t'.join(x))
weight4.collect()

['2003\tcouncil,1.431364;insurance,0.954243;welcomes,0.954243',
 '2004\tcowboys,0.954243;eels,0.954243;survive,0.954243',
 '2020\tcoronavirus,1.908485;china,0.954243;economy,0.954243']

In [44]:
weight4.saveAsTextFile('output1.txt')

Py4JJavaError: An error occurred while calling o1439.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:106)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:406)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1062)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationSco

In [6]:
sc.stop()